In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Conv1D, MaxPooling1D, Flatten, TimeDistributed




## reload scripts before executing them
%load_ext autoreload
%autoreload 2

## importing python scripts from the directores

from dataset.reduce_memory import reduce_mem_usage
from dataset.create_windows import create_windows
from dataset.create_submission import create_submission_csv

from evaluation.rmsse_loss import root_mean_squared_scaled_error

from train.cross_validate import cross_validate

from models.lstm import lstm_model
from models.cnn_lstm import cnn_lstm_model


import wandb
from wandb.keras import WandbCallback

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# read data

data_dir = 'data/'

train_sales = pd.read_csv(data_dir + 'sales_train_validation.csv')
#sell_prices = pd.read_csv(data_dir + 'sell_prices.csv')
calendar = pd.read_csv(data_dir + 'calendar.csv')
submission_file = pd.read_csv(data_dir + 'sample_submission.csv')
events_feature = pd.read_csv('eventencoding.csv')

In [4]:
train_sales = reduce_mem_usage(train_sales) # takes about 4mins

Mem. usage decreased to 95.00 Mb (78.7% reduction)


In [5]:
# create training data
sales = train_sales.drop(["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"], axis=1).T

# normalize
scaler = MinMaxScaler()
scaler.fit(sales)
sales = scaler.transform(sales)
sales = pd.DataFrame(sales)

In [6]:
# add extra features

# event feature
sales = pd.concat([sales, events_feature[:1913]], axis=1)

In [7]:
timesteps = 28
prediction_steps = 1
len_window = timesteps + prediction_steps

nr_training_days = sales.shape[0]
nr_sets = nr_training_days - len_window + 1

In [8]:
X, y = create_windows(sales, timesteps, prediction_steps, event_features=True)

In [10]:
# cnn-lstm model

n_features = X.shape[2]
n_outputs = y.shape[1]

In [14]:
lstm_model = cnn_lstm_model(timesteps,n_features, n_outputs,optimizer='adam', loss='mse')

In [15]:
# train & validate model (1 epoch takes about 1.5mins)

model.fit(X, y, batch_size=32, epochs=10, verbose=1)

Epoch 1/10
1885/1885 [==============================] - 50s 26ms/step - loss: 0.0158
Epoch 2/10
1885/1885 [==============================] - 39s 21ms/step - loss: 0.0135
Epoch 3/10
1885/1885 [==============================] - 35s 19ms/step - loss: 0.0131
Epoch 4/10
1885/1885 [==============================] - 35s 19ms/step - loss: 0.0128
Epoch 5/10
1885/1885 [==============================] - 36s 19ms/step - loss: 0.0126
Epoch 6/10
1885/1885 [==============================] - 36s 19ms/step - loss: 0.0125
Epoch 7/10
1885/1885 [==============================] - 35s 19ms/step - loss: 0.0124
Epoch 8/10
1885/1885 [==============================] - 34s 18ms/step - loss: 0.0123
Epoch 9/10
1885/1885 [==============================] - 36s 19ms/step - loss: 0.0122
Epoch 10/10
1885/1885 [==============================] - 37s 19ms/step - loss: 0.0122


In [ ]:
# test model

for i in range(28):    
    # get input for prediction by selecting last 28 days from sales
    X_pred = []
    X_pred.append(sales.iloc[-timesteps:].to_numpy())
    X_pred = np.array(X_pred)
    
    # get prediction
    prediction = model.predict(X_pred)
    
    # get event feature for predicted day
    events = events_feature.iloc[1913 + i].to_numpy()
    
    # add prediction+events to sales so that it can be used for next prediction
    sales.loc[sales.shape[0]] = np.append(prediction[0], events)
    
predictions = sales.iloc[-28:,:30490]
predictions = scaler.inverse_transform(predictions)
predictions = np.round(np.abs(predictions))
predictions = pd.DataFrame(predictions).T

In [ ]:
# create submission file

predictions_copy = predictions
final_submission = pd.concat([predictions, predictions_copy])
final_submission.reset_index(drop=True, inplace=True)
final_submission = final_submission.astype(int)
final_submission.insert(0, 'id', submission_file['id'])
final_submission.columns = ['id'] + [f"F{i}" for i in range(1, 29)]

final_submission.to_csv('submission_cnn.csv', index=False)